In [1]:
# Parameters
LAM = 0.001


In [2]:
for v in ['LAM']:
    if (v not in locals()) or (v not in globals()):
        raise TypeError(f"{v} not defined")
    else:
        print(f"{v} = {eval(v)}")

LAM = 0.001


In [3]:
import Spectra
import pickle

import numpy as np
np.random.seed(42)

#from Spectra import Spectra_gpu as spc_gpu

from pyprojroot import here

import scanpy as sc

import pandas as pd

import random
random.seed(42)

In [4]:
OVERLAP_THR = 0.3

# Load data
In the previous script we generated the following objects to be used as input for spectra: 
1. Processed adata in which we:
   - We included a column called *spectra_categories* as obs_key to be used for the celltypes definition.
   - We subsetted the adata object to contain the following genes:
     - (..)
     - (..): Differentially expressed genes per celltype for each disease *vs* healthy
     - Genes present in the Gene Set, which already included the immunologist list of genes
     
2. Gene set dictionary that contains:
   - Gene sets from the human immunology knowledge base (cytopus)
   - Gene sets from the SPECTRA paper
   - Immunologist list of genes  

In [5]:
# project_name = "SPECTRA_SEACell"
# project_dir = "{}/results/{}/".format(workDir, project_name)

project_dir = here('03_downstream_analysis/03_SPECTRA/')

In [6]:

adata = sc.read_h5ad(here('03_downstream_analysis/04_integration_with_annotation/results/normalized_adatas_nextflow/SEAcellAdataMerged.log1p.h5ad'))

### Load gene set dict
with open(here('03_downstream_analysis/05_SPECTRA/results/SPECTRA_GeneSet_ensg_v2.pickle'), 'rb') as f:
    gene_set_dict = pickle.load(f)

## Load selected gene
selected_genes = pd.read_pickle(here('03_downstream_analysis/02_gene_universe_definition/results/04_selected_gene_list.pkl'))

In [7]:
adata

AnnData object with n_obs × n_vars = 71108 × 8253
    obs: 'Level1', 'sampleID', 'disease'
    uns: 'log1p'

**Generate category `spectra_categories` in the adata object**

Modify celltypes categories to obtain the desired categories for downstream analysis

Here you should modify the annotations included in Most_abundant_celltype so that the categories match the annotations you want to run factors for. 

In [8]:
# define name for spectra column 
obs_key = "spectra_categories"

# Check current categories to see if they match with SPECTRA factors
print([c for c in np.sort(adata.obs['Level1'].unique())])
print(sorted(list(gene_set_dict.keys())))

['B', 'DC', 'ILC', 'Mono', 'Plasma', 'T_CD4_Naive', 'T_CD4_NonNaive', 'T_CD8_Naive', 'T_CD8_NonNaive', 'UTC', 'pDC']
['B', 'DC', 'ILC', 'Mono', 'Plasma', 'T_CD4_Naive', 'T_CD4_NonNaive', 'T_CD8_Naive', 'T_CD8_NonNaive', 'UTC', 'global', 'pDC']


In [9]:
# Apply the mapping function to create the 'spectra_categories' column
adata.obs[obs_key] = adata.obs['Level1']

# Modify Most_abundant_celltype so the categories match the annotations you want to run factors for. 
#adata.obs[obs_key] = adata.obs[obs_key].replace({'TCD4naive': 'Tnaive', #EXAMPLE
#                                                 'TCD8naive': 'Tnaive'  #EXAMPLE
#                                                })

# Check current categories to see if they match with SPECTRA factors
#print(sorted(set(adata.obs['MostAbundantCellType'])))
#print(sorted(list(gene_set_dict.keys())))

In [10]:
varDF = pd.read_csv(here('03_downstream_analysis/05_SPECTRA/results/varDF.csv'), index_col=0)
varDF

,hgnc_id,symbol,locus_group,HUGO_status,mt,ribo,hb,plt
ensembl_gene_id,,,,,,,,
ENSG00000000003,HGNC:11858,TSPAN6,protein_coding,official,False,False,False,False
ENSG00000000419,HGNC:3005,DPM1,protein_coding,official,False,False,False,False
ENSG00000000457,HGNC:19285,SCYL3,protein_coding,official,False,False,False,False
ENSG00000000460,HGNC:25565,FIRRM,protein_coding,official,False,False,False,False
ENSG00000000938,HGNC:3697,FGR,protein_coding,official,False,False,False,False
...,...,...,...,...,...,...,...,...
ENSG00000288253,NaN,ENSG00000288253,lncRNA,non-official,False,False,False,False
ENSG00000288302,NaN,LOC124903386,lncRNA,non-official,False,False,False,False
ENSG00000288398,NaN,ENSG00000288398,lncRNA,non-official,False,False,False,False


In [11]:
adata.var = adata.var.merge(varDF, left_index=True, right_index=True, how='left')
adata.var

,hgnc_id,symbol,locus_group,HUGO_status,mt,ribo,hb,plt
ensembl_gene_id,,,,,,,,
ENSG00000000003,HGNC:11858,TSPAN6,protein_coding,official,False,False,False,False
ENSG00000000457,HGNC:19285,SCYL3,protein_coding,official,False,False,False,False
ENSG00000000938,HGNC:3697,FGR,protein_coding,official,False,False,False,False
ENSG00000000971,HGNC:4883,CFH,protein_coding,official,False,False,False,False
ENSG00000001036,HGNC:4008,FUCA2,protein_coding,official,False,False,False,False
...,...,...,...,...,...,...,...,...
ENSG00000278817,NaN,ENSG00000278817,protein_coding,non-official,False,False,False,False
ENSG00000278828,HGNC:4775,H3C10,protein_coding,official,False,False,False,False
ENSG00000280670,HGNC:27003,CCDC163,protein_coding,official,False,False,False,False


**Load gene_universe for SPECTRA** 

In xxx(Davide to add) notebook we defined the list of *interesting genes*. These genes correspond to a list DEG per cell type obtained comparing healthy *vs* each disease.  

Here we load them to be included in the spectra_adata

-- if already subset the adata object after integration, then it is not needed

Ensure unwanted genes TCR/BCR/HLA/MT/RB are removed!

In [12]:
# Retrieve MT and RB genes present in the dataset
MT_gene_idx = adata.var["symbol"].str.startswith("MT-")
print(f"{np.sum(MT_gene_idx)} mitochondrial genes")

RB_gene_idx = adata.var["symbol"].str.startswith(("RPS", "RPL"))
print(f"{np.sum(RB_gene_idx)} ribosomal genes")

# Retrieve TCR and BCR present in the dataset
TCR_gene_idx = adata.var["symbol"].str.contains("^TRA(J|V)|^TRB(J|V|D)")
print(f"{np.sum(TCR_gene_idx)} TCR genes")

BCR_gene_idx = adata.var["symbol"].str.contains("^IGH(J|V)")
print(f"{np.sum(BCR_gene_idx)} IGH genes")

MHC_gene_idx = adata.var["symbol"].str.contains("^HLA-")
print(f"{np.sum(MHC_gene_idx)} MHC genes")

# ### TO MODIFY & RUN FROM STEP 4 (when RBC and Platelets will be filtered out)
HB_gene_idx = adata.var["symbol"].str.contains("^HB[^(P)]")
print(f"{np.sum(HB_gene_idx)} HB genes")

PLT_gene_idx = adata.var["symbol"].isin(["PPBP", "PDGF", "ANG1", "LAPTM4B", "WASF3", "TPM3", "PF4", "TAC1"])
print(f"{np.sum(PLT_gene_idx)} PLT genes")

0 mitochondrial genes
0 ribosomal genes
0 TCR genes
0 IGH genes
20 MHC genes
1 HB genes
3 PLT genes


/scratch_tmp/23789465/ipykernel_103685/134033282.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  TCR_gene_idx = adata.var["symbol"].str.contains("^TRA(J|V)|^TRB(J|V|D)")
/scratch_tmp/23789465/ipykernel_103685/134033282.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  BCR_gene_idx = adata.var["symbol"].str.contains("^IGH(J|V)")


#### Subsetting adata by keeping only selected genes

In [13]:
spectra_adata = adata

In [14]:
spectra_adata.layers = None
spectra_adata.obs = spectra_adata.obs[[obs_key]]

In [15]:
spectra_adata

AnnData object with n_obs × n_vars = 71108 × 8253
    obs: 'spectra_categories'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt'
    uns: 'log1p'

# Run Spectra

In [16]:
#Spectra_gpu
model = Spectra.est_spectra(
    adata=spectra_adata, 
    gene_set_dictionary=gene_set_dict,#spectra_gene_set, 
    use_highly_variable=False,
    cell_type_key=obs_key, 
    use_weights=True,
    lam=LAM, #varies depending on data and gene sets, try between 0.5 and 0.001 #Thomas Walle suggested to try: 0.01-0.1
    delta=0.001, 
    kappa=None,
    rho=0.001, 
    use_cell_types=True,
    n_top_vals=200,
    label_factors=True, 
    overlap_threshold=OVERLAP_THR,
    clean_gs = True, 
    verbose = True,
    min_gs_num = 2,
    num_epochs=15000
)

Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is now correctly formatted.


  0%|          | 0/15000 [00:00<?, ?it/s]

  0%|          | 1/15000 [00:17<72:10:11, 17.32s/it]

  0%|          | 2/15000 [00:32<67:57:57, 16.31s/it]

  0%|          | 3/15000 [00:50<69:51:31, 16.77s/it]

...

 26%|██▌       | 3873/15000 [16:48:41<49:11:03, 15.91s/it]

 26%|██▌       | 3874/15000 [16:48:57<49:20:32, 15.97s/it]

 26%|██▌       | 3875/15000 [16:49:14<50:24:57, 16.31s/it]

 26%|██▌       | 3876/15000 [16:49:31<51:08:47, 16.55s/it]

 26%|██▌       | 3876/15000 [16:49:46<48:18:02, 15.63s/it]

# Save adata with Spectra results

In [17]:
model.save(here(f'03_downstream_analysis/05_SPECTRA/results/04_spectra_model_compact_lam{LAM}_overlap{OVERLAP_THR}_v2'))

In [18]:
spectra_adata.write(here(f'03_downstream_analysis/05_SPECTRA/results/04_MAIN_SEAcell_scANVInorm_SPECTRA_lam{LAM}_overlap{OVERLAP_THR}_v2.h5ad'), compression="gzip")

In [19]:
adata = sc.read_h5ad(here("03_downstream_analysis/02_gene_universe_definition/results/04_MAIN_geneUniverse_noRBCnPlatelets.h5ad"), backed='r')